In [ ]:
!pip install --ignore-installed sentencepiece langchain huggingface_hub

In [ ]:
!pip install git+https://github.com/huggingface/accelerate.git

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, logging
import textwrap
import locale
from langchain import HuggingFacePipeline
from transformers import pipeline
from huggingface_hub import snapshot_download
from langchain import PromptTemplate, LLMChain
locale.getpreferredencoding = lambda: "UTF-8"
import torch
import transformers
import torch.nn as nn

In [ ]:
pwd

## Upload model on hf

In [43]:
# rm -r models--sambanovasystems--SambaLingo-Arabic-Base/

rm: cannot remove 'models--sambanovasystems--SambaLingo-Arabic-Base/': No such file or directory


In [ ]:
# model_id = "fireworks-ai/firefunction-v1"

In [ ]:
# model_path = snapshot_download(repo_id=model_id, cache_dir="/workspace/models/")

In [ ]:
model_path = "/workspace/axolotl/src/out"

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype = torch.bfloat16,
)

In [ ]:
model

In [ ]:
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_path, add_eos_token=False, trust_remote_code=True)
# tokenizer.add_tokens(['<sep>', '<pad>', '<mask>', '<predict>', '<FIM_SUFFIX>', '<FIM_PREFIX>', '<FIM_MIDDLE>','<commit_before>','<commit_msg>','<commit_after>','<jupyter_start>','<jupyter_text>','<jupyter_code>','<jupyter_output>','<empty_output>'], special_tokens=True)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.add_special_tokens({'bos_token':'<|im_start|>system'},)
# tokenizer.add_special_tokens({'eos_token':'<|im_end|>'})
# tokenizer.add_special_tokens({'pad_token': '<|im_end|>'})
# tokenizer.add_tokens(['<|im_start|>system', '<|im_start|>user','<|im_start|>assistant', '<<SYS>>', '[INST]', '[/INST]'], special_tokens=True)
print("Model & tokenizer loaded")

In [ ]:
tokenizer

In [ ]:
# model.push_to_hub("qxsecureserver/tool-selection", max_shard_size=1024, private=True)

In [ ]:
# tokenizer.push_to_hub("qxsecureserver/tool-selection", private=True)

## Check tokenizer

In [44]:
model_path = "01-ai/Yi-6B"
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_path, add_eos_token=False, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'bos_token':'<s>'},)
tokenizer.add_special_tokens({'eos_token':'</s>'})
tokenizer.add_special_tokens({'unk_token': '<unk>'})
tokenizer.add_tokens(['<|im_start|>', '<|im_start|>system', '<|im_start|>user','<|im_start|>assistant'], special_tokens=True)
print("Model & tokenizer loaded")

Model & tokenizer loaded


In [ ]:
# tokenizer

In [ ]:
# tokenizer.encode('<|im_start|>assistant')

In [ ]:
# input_ids = []

# labels = []

# tokenizer.decode(input_ids)

In [64]:
# input_ids = []

# labels = []

In [69]:
# count = 0
# for label in labels:
#     if label == -100:
#         count += 1
#     else:
#         break

# first_input_ids = input_ids[:count]
# rest_input_ids = input_ids[count:]

# # Initialize variables
# split_input_ids = [[]] # To store the resulting lists
# current_count = 0 # To count the consecutive labels
# last_label = -100
# # Loop through labels to determine splits
# index = 0
# for label in labels:
#     if label == -100:
#         if last_label != -100:
#             split_input_ids.append([])
#         split_input_ids[-1].append(input_ids[index])
#         last_label = label
#     else:
#         if last_label == -100:
#             split_input_ids.append([])
#         split_input_ids[-1].append(input_ids[index])
#         last_label = label
#     index += 1

In [ ]:
for split in split_input_ids:
    print(tokenizer.decode(split))
    print("\n\n------------------------------------")

In [ ]:
history = []
# history.append({'role': 'user', 'content': 'Who are you? and who made you?'})
# history.append({'role': 'assistant', 'content': 'I am AskQx AI Assistant, a chatbot designed and developed by the team at QXLABAI. I am built to help users with various tasks such as providing information, answering questions, and engaging in conversations.'})
# history.append({'role': 'user', 'content': 'What is the 6 most famous song of all time?'})
# history.append({'role': 'assistant', 'content': '''Determining the "most famous" or most popular songs can be subjective, but some well-known songs that have had significant cultural impact include:

# 1. "Bohemian Rhapsody" by Queen
# 2. "Imagine" by John Lennon
# 3. "Like a Rolling Stone" by Bob Dylan
# 4. "Billie Jean" by Michael Jackson
# 5. "Stairway to Heaven" by Led Zeppelin
# 6. "Smells Like Teen Spirit" by Nirvana

# These are just a few examples; there are countless other iconic songs from different genres and eras that could also be considered famous or influential.'''})
history.append({'role': 'user', 'content': 'Solve 25-4*2+3'})
history.append({'role': 'assistant', 'content': '''25 - 4 * 2 + 3 = 

First, perform the multiplication:

25 - 8 + 3

Now, subtract:

17 + 3 = 20

Finally, add the numbers:

20

So, the result of 25 - 4 * 2 + 3 is 20.'''})
# history.append({'role': 'user', 'content': 'Add 3 more in it'})
# history.append({'role': 'assistant', 'content': '7'})

In [ ]:
system = "You are AskQx AI Assistant developed by QXLABAI"

In [ ]:
def prompt_aptchat(prompt, history=history):
    new_prompt = f'''{tokenizer.bos_token}<|im_start|>system\n{system}<|im_end|>\n'''
    for item in history:
        if item['role'] == 'user':
            new_prompt += f"<|im_start|>user\n{item['content']}<|im_end|>\n"
        else:
            new_prompt += f"<|im_start|>assistant\n{item['content']}<|im_end|>\n"
    new_prompt += f"<|im_start|>user\n{prompt}<|im_end|>\n"
    new_prompt += f"<|im_start|>assistant\n"
    return new_prompt

In [ ]:
def prompt_aptchat2(prompt, history=history):
    new_prompt = f'''{tokenizer.bos_token}<|system|> {system}{tokenizer.eos_token}'''
    for item in history:
        if item['role'] == 'user':
            new_prompt += f"<|user|> {item['content']}{tokenizer.eos_token}"
        else:
            new_prompt += f"<|assistant|> {item['content']}{tokenizer.eos_token}"
    new_prompt += f"<|user|> {prompt}{tokenizer.eos_token}"
    new_prompt += f"<|assistant|> "
    return new_prompt

In [ ]:
def prompt_aptchat3(prompt, history=history):
    new_prompt = f'''{tokenizer.bos_token}<|im_start|>system\n{system}{tokenizer.eos_token}\n'''
    for item in history:
        if item['role'] == 'user':
            new_prompt += f"<|im_start|>user\n{item['content']}{tokenizer.eos_token}\n"
        else:
            new_prompt += f"<|im_start|>assistant\n{item['content']}{tokenizer.eos_token}\n"
    new_prompt += f"<|im_start|>user\n{prompt}{tokenizer.eos_token}\n"
    new_prompt += f"<|im_start|>assistant\n"
    return new_prompt

In [ ]:
# prompt_aptchat2('''add 2 in it''')

In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True)
stream_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4096,
    temperature=0.6,
    top_p=0.9,
    do_sample=True,
    top_k=50,
    repetition_penalty=1.15,
    batch_size=8,
    streamer=streamer,
)
stream_llm = HuggingFacePipeline(pipeline=stream_pipeline)
stream_template = f'''User: '''
stream_template += "{query}"
stream_template += f'''Assistant: '''

stream_prompt = PromptTemplate(template=stream_template, input_variables=["query"])
stream_llm_chain = LLMChain(prompt=stream_prompt, llm=stream_llm)

In [ ]:
stream_llm_chain.run(prompt_aptchat2('''Who are you? and who made you?'''))